In [1]:
%load_ext autoreload
%autoreload 2
%run A_simple_net.ipynb

Detected PyNN version 0.9.4 and Neo version 0.6.1
2020-12-08T11:50:50+00:00

CPython 3.6.9
IPython 7.15.0

numpy 1.19.0
matplotlib 3.2.2
pyNN 0.9.4
neo 0.6.1
nixio 1.4.9
pandas 1.1.4

compiler   : GCC 8.4.0
system     : Linux
release    : 4.4.0-87-generic
machine    : x86_64
processor  : x86_64
CPU cores  : 32
interpreter: 64bit
host name  : 96c3276e2566
Git hash   : 0f10f20007888cd754cfe22267f3b0af3ffa0347
Git repo   : https://github.com/SpikeAI/2020-11_brainhack_Project7
Git branch : main


In [2]:
%mkdir -p outputs

In [3]:
date = '20201208'

In [4]:
%mkdir -p Results/{date}/

# scanning over single parameters

## initializing parameters

In [5]:
tag = date + '_scan_' + simulator

In [6]:
buildCPUTime, simCPUTime, writeCPUTime = run(verbose=True)

2020-12-08 11:50:51 INFO: Read cfg files: /home/spinnaker/sPyNNaker/lib/python3.6/site-packages/spinn_front_end_common/interface/spinnaker.cfg, /home/spinnaker/sPyNNaker/lib/python3.6/site-packages/spynnaker/pyNN/spynnaker.cfg, /home/spinnaker/.spynnaker.cfg
2020-12-08 11:50:51 INFO: Will search these locations for binaries: /home/spinnaker/sPyNNaker/lib/python3.6/site-packages/spinn_front_end_common/common_model_binaries : /home/spinnaker/sPyNNaker/lib/python3.6/site-packages/spynnaker/pyNN/model_binaries


['/home/spinnaker/sPyNNaker/lib/python3.6/site-packages/spinn_front_end_common/interface/spinnaker.cfg', '/home/spinnaker/sPyNNaker/lib/python3.6/site-packages/spynnaker/pyNN/spynnaker.cfg', '/home/spinnaker/.spynnaker.cfg']


2020-12-08 11:50:51 WARNING: /home/spinnaker/2020-11_brainhack_Project7/benchmark/reports has 4 old reports that have not been closed
2020-12-08 11:50:51 WARNING: /home/spinnaker/2020-11_brainhack_Project7/benchmark/application_generated_data_files has 4 old reports that have not been closed
2020-12-08 11:50:51 WARNING: A timestep was entered that has forced sPyNNaker to automatically slow the simulation down from real time by a factor of 10. To remove this automatic behaviour, please enter a timescaleFactor value in your .spynnaker.cfg
2020-12-08 11:50:51 INFO: Setting time scale factor to 10.
2020-12-08 11:50:51 INFO: Setting machine time step to 100 micro-seconds.
2020-12-08 11:50:51 WARNING: Extra params {'threads': 1, 'filename': 'va.xml', 'label': 'VA'} have been applied to the setup command which we do not consider
2020-12-08 11:50:51 INFO: Simulating for 10000 0.1ms timesteps using a hardware timestep of 10us
2020-12-08 11:50:51 INFO: Starting execution process


0 Creating cell populations...
0 Initialising membrane potential to random values...
0 Connecting populations...
0 Setting up recording...
0 Running simulation...


2020-12-08 11:50:54 INFO: Time 0:00:03.256050 taken by SpallocMaxMachineGenerator
Pre allocating resources for Extra Monitor support vertices
|0%                          50%                         100%|
2020-12-08 11:51:03 INFO: Time 0:00:08.870947 taken by PreAllocateResourcesForExtraMonitorSupport
Partitioning graph vertices
|0%                          50%                         100%|
Partitioning graph edges
|0%                          50%                         100%|
2020-12-08 11:51:08 INFO: Time 0:00:05.333261 taken by PartitionAndPlacePartitioner
Created spalloc job 5544623
2020-12-08 11:51:08 INFO: Created spalloc job 5544623
Waiting for board power commands to complete.
2020-12-08 11:51:08 INFO: Waiting for board power commands to complete.
2020-12-08 11:51:13 INFO: Time 0:00:05.047560 taken by SpallocAllocator
2020-12-08 11:51:13 INFO: Creating transceiver for 10.11.195.129
2020-12-08 11:51:13 INFO: Working out if machine is booted
2020-12-08 11:51:17 INFO: Attempting t

0 Writing data to file...



Getting v for All Cells
|0%                          50%                         100%|


--- Vogels-Abbott Network Simulation ---
Nodes                  : 1
Number of Neurons      : 1000
Excitatory conductance : 4 nS
Inhibitory conductance : 51 nS
Excitatory rate        : 0.90625 Hz
Inhibitory rate        : 0.875 Hz
Build time             : 4.89283 ms
Simulation time        : 65749 ms
Writing time           : 237.887 ms


## Running Model and Loading Results

In [7]:
import os
import pandas as pd
verbose = True
def scan(variable, values, tag=tag, verbose=verbose):
    filename = 'outputs/{tag}_{variable}.json'.format(tag=tag, variable=variable)
    try:
        df = pd.read_json(filename)
        print('Loaded file')
    except:
        df = pd.DataFrame([], columns=[variable, 'buildCPUTime', 'simCPUTime', 'writeCPUTime'])
        for i, value in enumerate(values):
            if verbose: print('{variable=}---{value=}'.format(value=value, variable=variable))
            args = {}
            args[variable] = value
            buildCPUTime, simCPUTime, writeCPUTime = run(**args)
            df.loc[i]= {variable: value, 'buildCPUTime':buildCPUTime, 'simCPUTime':simCPUTime, 'writeCPUTime':writeCPUTime}
        df.to_json(filename)
    return df

In [8]:
import matplotlib.pyplot as plt

# https://stackoverflow.com/questions/31978948/python-stats-models-quadratic-term-in-regression
def plot(df, do_intercept=True):
    variable = df.columns[0]
    fig, ax = plt.subplots(figsize=(8,8))
    
    for regressor, color in zip(['buildCPUTime', 'simCPUTime', 'writeCPUTime'],
                                ['r', 'g', 'b']):

        import patsy
        if do_intercept:
            y, X = patsy.dmatrices('{regressor} ~ {variable}'.format(regressor=regressor, variable=variable), data=df, return_type='dataframe')
        else:
            y, X = patsy.dmatrices('{regressor} ~ 0 + {variable}'.format(regressor=regressor, variable=variable), data=df, return_type='dataframe')

        import statsmodels.api as sm
        fit = sm.OLS(y, X).fit()
        print(fit.summary())

        if do_intercept: intercept = fit.params.Intercept
        slope = fit.params[variable]

        values = np.array(df[variable])
        if do_intercept:
            ax.plot(values, intercept + slope*values, c=color, lw=2, label=f"{regressor}={slope:.3f}*{variable} + {intercept:.3f}")
        else:
            ax.plot(values, slope*values, c=color, lw=2, label=f"{regressor}={slope:.3f}*{variable}")
        df.plot(x=variable, y=regressor, c=color, lw=1, ls='-.', ax=ax, legend=False)

    ax.set_xlim(0)
    ax.set_ylabel('Time (ms)')
    ax.set_ylim(0)
    ax.legend()
    return fig, ax

## plot as a function of neuron numbers

In [9]:
import numpy as np
N_pops = np.array([int(k)**2 for k in np.linspace(8**2, 45**2, 14, endpoint=True)**.5])
N_pops

array([  64,  196,  361,  484,  625,  784,  961, 1089, 1225, 1369, 1521,
       1681, 1849, 2025])

In [ ]:
results_pop = scan('N_pop', N_pops)

In [ ]:
fig, ax = plot(results_pop)

In [ ]:
fig.savefig(tag + '_N_pop.png')

## plot as a function of simtime

In [ ]:
simtimes = np.linspace(500, 3000, 15, endpoint=True)
simtimes

In [ ]:
results_simtime = scan('simtime', simtimes)

In [ ]:
fig, ax = plot(results_simtime, do_intercept=True)      

In [ ]:
fig.savefig(tag + '_simtime.png')

# scanning over two parameters

## Setting-up model and initializing parameters

In [ ]:
tag = date + '_scan-2D_' + simulator

## Running Model and Loading Results

In [ ]:
variables = ['N_pop', 'simtime']
filename = 'outputs/' + tag
for variable in variables: filename +=  '_' + variable
filename += '.json'

In [ ]:
import os
import time
import pandas as pd
verbose = True
def scan(variables, values, tag=tag, verbose=verbose):
    filename = 'outputs/' + tag
    for variable in variables: filename +=  '_' + variable
    filename += '.json'
    try:
        df = pd.read_json(filename)
        print('Loaded file')
    except:
        df = pd.DataFrame([], columns=variables + ['buildCPUTime', 'simCPUTime', 'writeCPUTime'])
        for i, values_ in enumerate(values):
            if verbose: print('{variable}---{values_}'.format(values_=values_, variable=variable))
            args = {}
            for variable, value in zip(variables, values_):
                args[variable] = value
            buildCPUTime, simCPUTime, writeCPUTime = run(**args)
            df.loc[i]= dict(zip(variables + ['buildCPUTime', 'simCPUTime', 'writeCPUTime'], 
                                list(values_) + [buildCPUTime, simCPUTime, writeCPUTime]))

        df.to_json(filename)
        
    return df

## Running time as a function of neuron numbers and simtime

In [ ]:
import numpy as np

In [ ]:
N_trials = 200
N_pops = [int(k)**2 for k in np.linspace(15, 60, N_trials, endpoint=True)]
simtimes = np.linspace(200, 3000, N_trials, endpoint=True)

In [ ]:
np.random.seed(42)
np.random.shuffle(N_pops)
np.random.shuffle(simtimes)
values = zip(N_pops, simtimes)

In [ ]:
results = scan(variables=variables, values=values)
results

## analysis

In [ ]:
results_ext = results.copy()
results_ext['simtime_N_pop'] = results_ext['simtime'] * results_ext['N_pop']
results_ext

In [ ]:
variables_ext = ['N_pop', 'simtime_N_pop']
X = results_ext[variables_ext].values.reshape(-1, len(variables_ext))


https://aegis4048.github.io/mutiple_linear_regression_and_visualization_in_python

See https://patsy.readthedocs.io/en/latest/quickstart.html

In [ ]:
import patsy
import statsmodels.api as sm

for regressor in ['buildCPUTime', 'simCPUTime', 'writeCPUTime']:
    y = results_ext[regressor].values
    #print(variables_ext, X.shape, y.shape)
    y, X = patsy.dmatrices('{regressor} ~ 0 + N_pop + simtime_N_pop'.format(regressor=regressor), data=results_ext, return_type='dataframe')

    fit = sm.OLS(y, X).fit()
    fit.summary()

    print('{regressor} (ms) = {fit0:.3f} * N_pop + {fit1:.3f}/1000 * simtime (ms) * N_pop'.format(regressor=regressor, fit0=fit.params[0], fit1=fit.params[1]*1000))